In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 0.0, 1.0
p_train = np.random.uniform(p_low, p_high, (train_size, 2)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 2)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 2)).astype(np.float32)

## Exact Solver

In [5]:
from src.problem import msQuadratic
model = msQuadratic()

In [6]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    model.set_param_val({"p":p})
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:12<00:00, 13.78it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.301369          0.287015      0.071896
std       0.015466          0.665298      0.030828
min      -0.310000          0.000000      0.057077
25%      -0.310000          0.000000      0.061129
50%      -0.310000          0.000000      0.061881
75%      -0.300000          0.000000      0.062861
max      -0.212920          3.460264      0.228069
